# Optimizers study 

In this file, we test optimizers performance over the baseline neural network model. 

Final results :

    Optimizer: SGD with momentum
    AUC score: 0.950
    Normalization: Yes and No
    


# 0. Import

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from statistics import mean

from torch.utils.data import DataLoader
import torch

from helpers.data_generation.file_management import read_hdf5
from helpers.data_generation.error_generation_chi2 import Residual, CombineDataset
from helpers.model.helpers_model import NeuralNet

import warnings
warnings.filterwarnings('ignore') 

## 1. Data generation

We generate the data with the following features. We remove noise-like and obvious error maps from the data by adding $MSE$ image metric and a limitation on the amplitude of the dataset.

* Parameters : 
    * Percentages of error : 
        [0.5\% 1.5\% 0.5\%] 
    * Ratio : 
        75\% 
    * Minimum mean square error : 
        $\chi^2_{low} > 1.2$ 
    * Maximum absolute pixel amplitude : 
        $|g(n,m)| < 6$
        
        
Where $g$ is an image and $g(n,m)$ is the pixel in line $n$ and column $m$.


In [ ]:
ratio = 0.75
percent = np.array([0.005, 0.015, 0.005])
size = 6000

batch_size = 50
max_epoch = 150

opti_name = np.array(['SGD', 'Adam', 'SGD/momentum'])



res = Residual()
res.build(size, ratio = ratio, per_error = percent)

str_ID =  "S"+str(size)+"R"+str(int(ratio*100))
[final_array, metadata] = read_hdf5(str_ID)
metadata ['ID'] = np.arange(0,final_array.shape[0])

## 2. Epoch and AUC optimizer selection 

This section focuses on the time of learning versus the model efficiency. 
* SGD :
    * Normalized : 
        Epoch : 146 - AUC : 0.935
    * Not normalized : 
        Epoch : 143 - AUC : 0.934
* SGD/momentum ;
    * Normalized : 
        Epoch : 57 - AUC : 0.945
    * Not normalized : 
        Epoch : 124 - AUC : 0.946
* Adam :
    * Normalized : 
        Epoch : 22 - AUC : 0.950
    * Not normalized : 
        Epoch : 91 - AUC : 0.939

In [ ]:
plt.figure()
for norm in np.array([True, False]):
    data_set = CombineDataset(metadata,'ID','class',final_array,norm)

    data_train, data_test = train_test_split(data_set,train_size=0.9,random_state=42)


    loader_train = DataLoader(data_train, batch_size = batch_size, 
                            num_workers = 4, drop_last=True, pin_memory = True)

    loader_test = DataLoader(data_test, batch_size = batch_size, 
                            num_workers = 4, drop_last=True, pin_memory = True)


    for opti in opti_name:
        print('----------------------------------------------------------------------------------')

        test_SGDnorm = np.zeros(max_epoch)
        netbasic = NeuralNet('BasicCNN', opti)
        for epoch in range(max_epoch):
            netbasic.train(loader_train)
            res = netbasic.test(loader_test, verbose = False)
            test_SGDnorm[epoch] = res
        
        if norm:
            txt = "Finished Training- Normalized: "+ opti +" - epoch: {:.3f} - auc: {:.3f} \n" 
            print(txt.format(np.argmax(netbasic.epoch_metric)+1, netbasic.max_met))
            plt.plot(np.arange(1, max_epoch+1), test_SGDnorm, label='Norm '+opti)
        else:
            txt = "Finished Training- Not normalized: "+ opti +" - epoch: {:.3f} - auc: {:.3f} \n" 
            print(txt.format(np.argmax(netbasic.epoch_metric)+1, netbasic.max_met))
            plt.plot(np.arange(1, max_epoch+1), test_SGDnorm, label=opti)
plt.legend()
plt.grid()
plt.savefig('figures/baseline/optimizers.jpeg')
plt.show()

## 4. K-fold optimizer selection

To select the optimizer of our problem, we perform a 5-fold approach to make sure of our final optimizer selection. 

* SGD/momentum ;
    * normalized data AUC : 0.950
    * not normalized data AUC : 0.950
* Adam :
    * normalized data AUC : 0.948
    * not normalized data AUC : 0.949
* SGD :
    * normalized data AUC : 0.943
    * not normalized data AUC : 0.938

In [ ]:
metadata ['ID'] = np.arange(0,final_array.shape[0])

data_set = CombineDataset(metadata,'ID','class',final_array)

k_folds = 5 
kfold = KFold(n_splits = k_folds, shuffle = True)


for norm in np.array([False, True]):
    data_set = CombineDataset(metadata,'ID','class',final_array, norm)
    for opti in opti_name:
        max_AUC = []
        for fold, (train_ids, test_ids) in enumerate(kfold.split(data_set)):
            print(f'FOLD {fold}')
            print('----------------------')

            train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
            test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

            loader_train = DataLoader(data_set, batch_size = batch_size, sampler = train_subsampler, num_workers = 4, pin_memory = True)
            loader_test = DataLoader(data_set, batch_size = batch_size, sampler = test_subsampler, num_workers = 4, pin_memory = True)
    

            netbasic = NeuralNet('BasicCNN', opti)
            while netbasic.current_epoch < max_epoch:
                netbasic.train(loader_train)
                res = netbasic.test(loader_test, verbose = False)
            max_AUC.append(netbasic.max_met)
            
        print('----------------------------------------------------------------------------------')
        if norm:
            txt = "Finished Training- Normalized: "+ opti +" - auc: {:.3f} \n" 
            print(txt.format(mean(max_AUC)))
        else:
            txt = "Finished Training- Not normalized: "+ opti +" - auc: {:.3f} \n" 
            print(txt.format(mean(max_AUC)))